In [ ]:
import os

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
try:
    import open3d as o3d
except ImportError:
    import sys
    print(sys.exc_info())
import pandas as pd
from scipy import interpolate

from dosipy.utils.integrate import elementwise_dblquad
from dosipy.utils.viz import scatter_2d, scatter_3d
from utils import load_processed_data, export_rect_idx

In [ ]:
# extract data

antenna = 'ArrayHorizontal'
distance = 10

df = load_processed_data(antenna, distance)
xyz = df[['x', 'y', 'z']].values
n = df[['nx', 'ny', 'nz']].values
pd = df['PD'].values

fig, ax = scatter_3d({'x': xyz[:, 0],
                      'y': xyz[:, 1],
                      'z': xyz[:, 2],
                      'pd': pd}, elev=[10], azim=[150])
ax.set_title(f'{antenna} at {distance} mm');

In [ ]:
# extract points visible from the EMF incidence POV (x direction)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
diameter = np.linalg.norm(
    pcd.get_max_bound() - pcd.get_min_bound()
)
radius = 100
center = pcd.get_center()
camera = [center[0] - diameter, center[1], center[2]]
_, mask = pcd.hidden_point_removal(camera, radius)
xyz_yz = xyz[mask]
pd_yz = pd[mask]
n_yz = n[mask]

fig, ax = scatter_3d({'x': xyz_yz[:, 0],
                      'y': xyz_yz[:, 1],
                      'z': xyz_yz[:, 2],
                      'pd': pd_yz}, elev=[10], azim=[180])
ax.set_title(f'{antenna} at {distance} mm');

In [ ]:
# find an appropriate domain

ps_pd_yz_idx = np.where(pd_yz == pd_yz.max())[0][0]
avg_center = [xyz_yz[ps_pd_yz_idx, 1], xyz_yz[ps_pd_yz_idx, 2]]
edge_length = 0.02
area = edge_length ** 2
origin, _ = export_rect_idx(xyz=xyz_yz,
                            center=avg_center,
                            edge_length=edge_length,
                            view='yz')

fig, ax = scatter_2d({'y': xyz_yz[:, 1],
                      'z': xyz_yz[:, 2],
                      'pd': pd_yz}, s=0.5)
ax.plot(*avg_center, 'x', ms=10, c='r')
patch_rect = Rectangle(origin, edge_length, edge_length, fc='none', ec='r', lw=2)
ax.add_patch(patch_rect)
ax.invert_xaxis()

In [ ]:
# find the max APDav and the corresponding averaging surface

ps_pd_yz_idx = np.where(pd_yz == pd_yz.max())[0][0]
av_dict = {'origin': [], 'xyz': [], 'apd': []}

for yc in np.linspace(center[1] - edge_length / 3, center[1] + edge_length / 3):
    for zc in np.linspace(center[2] - edge_length / 3, center[2] + edge_length / 3, 21):
        _origin, _idx_rect = export_rect_idx(xyz=xyz_yz,
                                             center=[yc, zc],
                                             edge_length=edge_length,
                                             view='yz')
        _xyz_rect = xyz_yz[_idx_rect]
        _pd_rect = pd_yz[_idx_rect]
        _n_rect = n_yz[_idx_rect]
        _apd = elementwise_dblquad(points=np.c_[_xyz_rect[:, 2], _xyz_rect[:, 1]],
                                   values=_pd_rect,
                                   degree=31,
                                   interp_func=interpolate.LinearNDInterpolator, fill_value=0
                                  ) / area  # area should actually be an actual surface area
        av_dict['origin'].append(_origin)
        av_dict['xyz'].append(_xyz_rect)
        av_dict['apd'].append(_apd)

ps_idx = np.argmax(res['apd'])
ps_apd = res['apd'][ps_idx]
ps_origin = res['origin'][ps_idx]
fig, ax = scatter_2d({'y': xyz_yz[:, 1],
              'z': xyz_yz[:, 2],
              'pd': pd_yz}, s=0.5)
ax.plot(*avg_center, 'x', ms=10, c='r')
ax.set_title(f'APD = {ps_apd:.2f} W/m2')
patch_rect = Rectangle(ps_origin, edge_length, edge_length, fc='none', ec='r', lw=2)
ax.add_patch(patch_rect)
ax.invert_xaxis()